## Loading and visualizing DICOM

This notebook demonstrates importing images in DICOM format to CERR's planC data structure and visualizing the imported objects. Phantom dataset made available by Lambin et al via CancerData and re-distributed along with the CERR package is used in this demonstration. Each patient daatset consists of CT scan for Lung and tumor segmentation.

#### Install pyCERR (development version)

In [ ]:
%%capture
pip install git+https://github.com/cerr/pyCERR/

#### Get a list of paths to patient datasets

In [7]:
from cerr import datasets
import os
phantom_dir = os.path.join(os.path.dirname(datasets.__file__),'radiomics_phantom_dicom')
pat_names = ['PAT1', 'PAT2', 'PAT3', 'PAT4']
all_pat_dirs = [os.path.join(phantom_dir, pat) for pat in pat_names]

#### Import contents of a patient directory to planC

In [12]:
# Import dicom to planC
from cerr import plan_container as pc
dcm_dir = r"L:\Data\RTOG0617\DICOM\NSCLC-Cetuximab_flat_dirs\0617-381906_09-09-2000-87022"
planC = pc.load_dcm_dir(dcm_dir)

('0617-381906', '0617-381906', '1.3.6.1.4.1.14519.5.2.1.6329.6468.105550208905127913957193987022', '1.3.6.1.4.1.14519.5.2.1.6329.6468.120543785947519379903473848716', 'RTSTRUCT', '', '', '', '', '', '')
('0617-381906', '0617-381906', '1.3.6.1.4.1.14519.5.2.1.6329.6468.105550208905127913957193987022', '1.3.6.1.4.1.14519.5.2.1.6329.6468.261810437325837391003704923226', 'RTDOSE', '', '', '', '', '', '')
('0617-381906', '0617-381906', '1.3.6.1.4.1.14519.5.2.1.6329.6468.105550208905127913957193987022', '1.3.6.1.4.1.14519.5.2.1.6329.6468.287225056317095442370425944994', 'RTPLAN', '', '', '', '', '', '')
('0617-381906', '0617-381906', '1.3.6.1.4.1.14519.5.2.1.6329.6468.105550208905127913957193987022', '1.3.6.1.4.1.14519.5.2.1.6329.6468.306596930655620435767854175882', 'CT', '', '', '', '', '', '')


C:\Users\aptea\Miniconda3\envs\testcerr\lib\site-packages\pydicom\valuerep.py:443: UserWarning: Invalid value for VR IS: '10.000'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)


#### Visualize scan and lung-tumor segmentation

Napari package is used for visualization. Napari requires a Display for rendering graphics and does not support visualization on a remote server (e.g. Google Colab)

In [13]:
from cerr import viewer as vwr
scanNum = [0]
doseNum = [0]
strNum = [5,len(planC.structure)]
displayMode = '2d' # '2d - path' or '3d - surface'
viewer, scan_layer, dose_layer, struct_layer = \
vwr.show_scan_struct_dose(scanNum, strNum, doseNum, planC, displayMode)

C:\Users\aptea\Miniconda3\envs\testcerr\lib\site-packages\napari\layers\shapes\_shapes_utils.py:627: RuntimeWarning: invalid value encountered in cast
  y = np.sign(x).astype(int)


#### Adjust Visualization settings

Properties of Napari viewer and image "layers" can be readily accessed and updated. Below cell shows an example of changing opacity, contrast ranges and gamma scale.

In [10]:
scan_layer[0].opacity = 1
scan_layer[0].contrast_limits_range = [-1000, 300]
scan_layer[0].contrast_limits = [-900, 200]
scan_layer[0].gamma = 0.7

#### Programmatically capturing screenshots

Metadata can be accessed from planC to update the Viewer to programmatically capture image slices and objects of interest. The example below shows screenshot capture of tumor segmentation on central CT slice

In [11]:
from skimage.io import imsave, imshow
import os
import cerr.contour.rasterseg as rs
import numpy as np
save_dir = r'L:\Aditya\batch_images'
# Get index of central axial slice
scanNum = 0 # index of scan in planC.scan
strNum = 0 # index of structure in planC.structure
mask3M = rs.getStrMask(strNum, planC)
rV, cV, sV = np.where(mask3M)
midSliceInd = int(np.round(sV.mean()))
# update viewer to display the central slice and capture screenshot
xV, yV, zV = planC.scan[scanNum].getScanXYZVals()
viewer.dims.set_point(2, zV[midSliceInd])
screenshot = viewer.screenshot()
capture_fname = os.path.join(save_dir,pat_names[0] + '.png')
imsave(capture_fname, screenshot)
print('saved to '+ capture_fname)

saved to L:\Aditya\batch_images\PAT1.png


#### Show the saved image

In [ ]:
im = imshow(capture_fname)
im.axes.set_axis_off()